사전작업

In [ ]:
###########################################
# 0. Huggingface login

# READ token
!huggingface-cli login --token  # your code

In [ ]:
###########################################
# 1-1. 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [5]:
###########################################
# 1-2. 프로젝트 폴더 만들기

import os
os.makedirs('/content/drive/MyDrive/Sarcasm-LLM', exist_ok=True)

In [ ]:

##########################################
# 1-3. 모듈 다운

!pip install bitsandbytes
!pip install accelerate
!pip install appdirs
!pip install loralib
!pip install black black[jupyter]
!pip install datasets
!pip install fire
!pip install git+https://github.com/huggingface/peft
!pip install transformers
!pip install sentencepiece sentence_transformers
!pip install scipy numpy scikit-learn pandas
!pip install -U datasets


In [ ]:
###########################################
# 1-4. 설치된 모듈 리스트 확인하기
!pip check
!pip list

In [8]:
import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
###########################################
# 2-1. 모듈 불러오기

import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
#from transformers import LlamaForCausalLM, LlamaTokenizer # These are not used, so they can be removed
#from transformers import AutoModelForCausalLM, AutoTokenizer # These are not used, so they can be removed
from transformers import AutoModelForSequenceClassification, AutoTokenizer # Import the correct model for Sequence Classification

from datasets import load_dataset

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training, #prepare_model_for_int8_training
    set_peft_model_state_dict
)

from peft import PeftModel
# %% [markdown]
# BERT 불러오기
# %%
device = 'auto'  #@param {type: "string"}
base_LLM_model = 'skt/kobert-base-v1'  #@param {type: "string"}
# %% [markdown]
# BERT는 생성 모델 아니라 다른 라이브러리
# %%
from transformers import AutoTokenizer, AutoModelForSequenceClassification # Import the correct model

# Change AutoModel to AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    base_LLM_model,
    torch_dtype=torch.float16,
    device_map=device,
    num_labels=2 # Assuming binary classification for Sarcasm/Non-Sarcasm
)

tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)

BERT 불러오기

In [10]:
device = 'auto'  #@param {type: "string"}
base_LLM_model = 'skt/kobert-base-v1'  #@param {type: "string"}

BERT는 생성 모델 아니라 다른 라이브러리

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

base_LLM_model = 'skt/kobert-base-v1'

tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)
model = AutoModelForSequenceClassification.from_pretrained(
    base_LLM_model,
    num_labels=2  # 이진 분류라면 2
)
# ✅ 필수: special token 추가 + embedding 리사이즈
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

In [12]:
###########################################
# 3-2. BOS, EOS, PAD 토큰 확인

bos = tokenizer.cls_token_id
eos = tokenizer.sep_token_id
pad = tokenizer.pad_token_id

In [ ]:
bos = tokenizer.cls_token_id    # 문장 시작: [CLS]
eos = tokenizer.sep_token_id    # 문장 끝: [SEP]
pad = tokenizer.pad_token_id    # 패딩: [PAD]

# 패딩 토큰이 없거나 EOS 토큰과 같다면 기본값 0으로 설정
if (pad is None) or (pad == eos):
    tokenizer.pad_token_id = 0

tokenizer.padding_side = "right"

print("BOS token:", bos)
print("EOS token:", eos)
print("PAD token:", tokenizer.pad_token_id)
print("length of tokenizer:", len(tokenizer))  # 예: 8002 (KoBERT의 vocab 크기)


In [ ]:
print(model)
print(type(model)) # 모델의 타입 확인

하이퍼파라미터는 자세히 수정 필요함

In [15]:
###########################################
# 5-1. 하이퍼 파라미터

# 데이터셋과 훈련 횟수와 관련된 하이퍼 파라미터
batch_size = 32
num_epochs = 1
micro_batch = 1 #GPU가 2대 이상일 때 활용하면 좋다..?
gradient_accumulation_steps = batch_size // micro_batch #GPU가 2대 이상일 때 활용하면 좋다..?

# 훈련 방법에 대한 하이퍼 파라미터
cutoff_len = 512
lr_scheduler = 'cosine'
warmup_ratio = 0.06 # warmup_steps = 100
learning_rate = 9.5e-4# 4e-4
optimizer = 'adamw_torch'
weight_decay = 0.01
max_grad_norm = 1.0 # Default값, 모델이 과적합되거나 잘못 된거 같으면 좀 줄여주면 좋음


# Tokenizer에서 나오는 input값 설정 옵션
# train_on_inputs = False
# add_eos_token = False # 데이터 셋에 관한 부분

# Others
resume_from_checkpoint = './sarcasm_LLM/checkpoint-001' # !! 만약 모델을 이어서 훈련하고 싶다면, './custom_LLM/checkpoint-[xxx]'와 같이 파일 경로를 입력해야 합니다!
# 모델 돌다가 다운되는거 방지
output_dir = './custom_LLM'

데이터셋 불러오기 - 코코사

In [16]:
#@title 🤗 Choose Dataset
dataset = 'YuminKim/KoCoSa' #@param {type: "string"}

#kyujinpy/Open-platypus-Commercial

In [ ]:
###########################################
# 5-1. 데이터셋 다운로드

from datasets import load_dataset

data = load_dataset(dataset)
print(data['train']) # 개수: 19079

In [ ]:
print(data['validation'])
print(data['test'])

템플릿 설정

In [19]:
template = {
    "prompt_sarcasm_explanation": (
        "아래는 한국어 상황 'context'를 보고, 발화(말, 댓글 등)'response'에 대한 풍자 여부를 판별하는 예제입니다. "
        "풍자를 판별하고 이를 설명하는 답변을 작성해주세요.\n\n"
        "### 상황:\n{context}\n\n"
        "### 발화:\n{response}\n\n"
        "### 답변:\n"
    ),
    "prompt_no_sarcasm_explanation": (
        "아래는 발화에 대한 상황을 추측해보고, 발화(말, 댓글 등)'response'에 대한 풍자 여부를 판별하는 예제입니다. "
        "풍자를 판별하는 답변만 작성해주세요.\n\n"
        "### 발화:\n{response}\n\n"
        "### 답변:\n"
    ),
    "response_split": "### 답변:"
}

In [ ]:
from typing import Optional

# 새 sarcasm 프롬프트 템플릿
template = {
    "prompt_sarcasm_explanation": (
        "아래는 한국어 상황 'context'를 보고, 발화(말, 댓글 등)'response'에 대한 풍자 여부를 판별하는 예제입니다. "
        "풍자를 판별하고 이를 설명하는 답변을 작성해주세요.\n\n"
        "### 상황:\n{context}\n\n"
        "### 발화:\n{response}\n\n"
        "### 답변:\n"
    ),
    "prompt_no_sarcasm_explanation": (
        "아래는 발화에 대한 상황을 추측해보고, 발화(말, 댓글 등)'response'에 대한 풍자 여부를 판별하는 예제입니다. "
        "풍자를 판별하는 답변만 작성해주세요.\n\n"
        "### 발화:\n{response}\n\n"
        "### 답변:\n"
    ),
    "response_split": "### 답변:"
}

class Prompter(object):
    """
    context, response 를 입력으로 받고,
    optional label 과 optional explanation 을 target 출력으로 묶어
    prompt 를 생성합니다.
    """
    def __init__(self, verbose: bool = False):
        self.template = template
        self.verbose = verbose

    def generate_prompt(
        self,
        context: str,
        response: str,
        label: Optional[str] = None,
        explanation: Optional[str] = None
    ) -> str:
        # explanation 이 있으면 분류+설명 템플릿 사용
        if explanation is not None:
            prompt = self.template["prompt_sarcasm_explanation"].format(
                context=context,
                response=response
            )
            if label:
                prompt += f"\n{label}"
            prompt += f"\n{explanation}"
        # explanation 이 없으면 분류만 템플릿 사용
        else:
            prompt = self.template["prompt_no_sarcasm_explanation"].format(
                response=response
            )
            if label:
                prompt += f"\n{label}"

        if self.verbose:
            print("Generated prompt:\n", prompt)
        return prompt

    def get_response(self, output: str) -> str:
        """
        모델 출력에서 '### 답변:' 뒤에 나오는 실제 답변만 추출합니다.
        """
        parts = output.split(self.template["response_split"])
        return parts[-1].strip() if len(parts) > 1 else output.strip()
# 사용 예시
prompter = Prompter(verbose=True)

# 1) label만 있을 때
p1 = prompter.generate_prompt(
    context="A: ... 영화 보셨나요?",
    response="B: 어, 인셉션은 꿈 이야기가 아니구요.",
    label="Non-Sarcasm"
)

# 2) label + explanation 있을 때
p2 = prompter.generate_prompt(
    context="A: ... 연애 경험 어때요?",
    response="B: 저한테도 소개팅은 첨이었어요.",
    label="Sarcasm",
    explanation="B는 소개팅 경험이 없다는 당연한 사실을 과장해 풍자로 표현했습니다."
)

print(p1)
print("—" * 40)
print(p2)

In [23]:
def generate_and_tokenize_prompt(example):
    full_input = prompter.generate_prompt(
        context=example["context"],
        response=example["response"],
        label=example["label"],
        explanation=example.get("sarcasm_explanation")
    )

    encoded = tokenize(full_input)

    label = example["label"]
    if isinstance(label, list):
        label = label[0]
    if isinstance(label, str):
        label = 1 if label.lower() == "sarcasm" else 0

    encoded["labels"] = int(label)
    return encoded


In [25]:
def tokenize(prompt: str):
    encoded = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=cutoff_len,
        return_tensors="pt"
    )
    return {k: v.squeeze(0).tolist() for k, v in encoded.items()}


In [ ]:
###########################################
# 5-5. 훈련 셋 만들기 (~3분)

val_data = None

train_data = data["train"].shuffle()
train_data = train_data.map(
    generate_and_tokenize_prompt,
    batched=False  # 중요!
)


In [ ]:
train_data = data["train"].shuffle(seed=42)

# train, validation, test 에 매핑
train_data = train_data.map(
    generate_and_tokenize_prompt,
    remove_columns=data["train"].column_names,
    batched=False
)

val_data = data["validation"].map(
    generate_and_tokenize_prompt,
    remove_columns=data["validation"].column_names,
    batched=False
)

test_data = data["test"].map(
    generate_and_tokenize_prompt,
    remove_columns=data["test"].column_names,
    batched=False
)

# 확인
print(train_data)
print(val_data)
print(test_data)


In [ ]:
pip install -U transformers

In [30]:
from transformers import DefaultDataCollator, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

data_collator = DefaultDataCollator(return_tensors="pt")


# 2. Trainer 정의
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=1,
        warmup_ratio=warmup_ratio,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        fp16=torch.cuda.is_available(),  # ✅ GPU 환경 확인
        logging_steps=10,
        optim="adamw_torch",
        save_strategy="steps",
        save_steps=50,
        save_total_limit=2,
        max_grad_norm=max_grad_norm,
        lr_scheduler_type=lr_scheduler,
        output_dir=output_dir,
        load_best_model_at_end=False,
        ddp_find_unused_parameters=False,
        group_by_length=False,
        remove_unused_columns=True
    ),
    data_collator=data_collator
)


In [31]:
# 데이터셋을 tokenizer 수정 이후에 다시 인코딩해야 함
train_data = data["train"].shuffle(seed=42).map(
    generate_and_tokenize_prompt,
    remove_columns=data["train"].column_names,
    batched=False
)

val_data = data["validation"].map(
    generate_and_tokenize_prompt,
    remove_columns=data["validation"].column_names,
    batched=False
)

test_data = data["test"].map(
    generate_and_tokenize_prompt,
    remove_columns=data["test"].column_names,
    batched=False
)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

base_LLM_model = "skt/kobert-base-v1"

tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)

model = AutoModelForSequenceClassification.from_pretrained(
    base_LLM_model,
    num_labels=2
)

# 반드시 tokenizer에 special token 추가
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # 이거 후에 절대 model 재선언 X


In [ ]:
###########################################
# 7-3. Training (fine-tuning)

## 훈련시간이 많이 소요됩니다! (~12h)
## 만약 중간에 colab이 끊긴다면, checkpoint를 이용해서 다시 훈련하세요!
### 만약 개인적인 GPU 자원이 있다면, 개인 서버나 컴퓨터에서 훈련시키는 것을 추천드립니다!
resume_from_checkpoint = None
torch.cuda.empty_cache()

trainer.train(resume_from_checkpoint=resume_from_checkpoint)